In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

In [ ]:
#Necessary Libraries

import numpy as np
import tensorflow
from keras.utils import np_utils
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
zip_path = "/content/drive/MyDrive/Dataset /nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip

In [ ]:
#Installing the wandb library
!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.login()

In [ ]:
# Storing the training and testing directories
import os 
Tags = ['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']
no_class = 10
train_dir='inaturalist_12K/train/'
test_dir='inaturalist_12K/val/'

In [ ]:
#Visualizing images of each class 

import matplotlib.pyplot as plt
import cv2 
import matplotlib.image as mpimg
fig = plt.figure(figsize=(20,8))
i=1
for tag in Tags:
  dir=os.path.join(train_dir,tag)
  image_name=os.listdir(dir)[0]
  image_path=os.path.join(dir,image_name)
  img=mpimg.imread(image_path)
  img_req=cv2.resize(img,(128,128)) 
  fig.add_subplot(2,5,i)
  plt.imshow(img_req)
  plt.axis('off')
  plt.title(tag)
  i+=1   


In [ ]:
# Prepare the dataset for training
def train_val__test_data_generation(batch_size=256,augment_data=False):

    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=45,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=False)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(128,128), batch_size=batch_size, subset="training")
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(128,128), batch_size=batch_size, subset="validation")
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(128,128), batch_size=batch_size)
    
    return train_generator, val_generator, test_generator

In [ ]:
train_generator, val_generator, test_generator = train_val__test_data_generation(batch_size=64,augment_data=True)

In [ ]:
#Build a small CNN model consisting of 5 convolution layers.

def CNN(filters,filter_size, image_size=128,
              dropout=0.2,batch_norm=False, dense_size=64, 
              regpara=0, no_of_classes=10, activation='relu'):

    model = Sequential()
    for i in range(5):
        if(i==0):
            model.add(Conv2D(filters=filters[i], kernel_size=filter_size[i], padding = 'same', input_shape = (image_size, image_size, 3),
                             kernel_regularizer= regularizers.l2(regpara)))
        else:
            model.add(Conv2D(filters=filters[i], kernel_size=filter_size[i], padding = 'same',
                             kernel_regularizer= regularizers.l2(regpara)))
        
        model.add(Activation(activation))

        if batch_norm:
            model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2,2)))

    
    # FC layer
    model.add(Flatten())
    model.add(Dense(dense_size, activation="relu",kernel_regularizer= regularizers.l2(regpara)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())

    #Output Layer
    model.add(Dense(no_of_classes, activation = "softmax"))

    return model

In [ ]:
sweep_config = {
    'method': 'bayes', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[(2,2),(2,2),(2,2),(2,2),(2,2)], [(3,3),(3,3),(3,3),(3,3),(3,3)],[(6,6),(5,5),(4,4),(3,3),(2,2)],[(2,2),(3,3),(4,4),(5,5),(6,6)]]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.005]
        },
        'dropout': {
            'values': [0, 0.2, 0.3, 0.4]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'activation': {
            'values': ['relu','selu','elu']
        },
        'batch_norm':{
            'values': [True,False]
        },
        'filters':{
            'values': [[32,32,32,32,32],[32,64,128,256,512],[32,16,8,4,2],[512,256,128,64,32]]
        },
        'augment_data': {
            'values': [True,False]
        },
        'batch_size': {
            'values': [32, 64, 128, 256]
        },
        'dense_size':{
            'values': [64, 128, 256, 512]
        }
    }
}

In [ ]:
def train():
    
    config_defaults = {
        'kernel_size': [(3,3),(3,3),(3,3),(3,3),(3,3)],
        'weight_decay': 0.005,
        'dropout': 0.2,
        'learning_rate': 1e-3,
        'activation': 'relu',
        'batch_size': 64,
        'epochs': 10,
        'batch_norm': True,
        'filters' : [32,32,32,32,32],
        'augment_data': True,
        'dense_size': 256,
        'seed': 1234,
        'no_of_classes': 10
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs

    config = wandb.config
    wandb.run.name = 'dense_size_'+ str(config.dense_size)+'_bs_'+str(config.batch_size)+'_ac_'+ config.activation

    #tf.keras.backend.clear_session()

    model=CNN(filters=config.filters,filter_size=config.kernel_size, image_size=128,
              dropout=config.dropout,batch_norm=config.batch_norm, dense_size=config.dense_size, 
              regpara=config.weight_decay, no_of_classes=config.no_of_classes, activation=config.activation )
    
    optimizer = Adam(learning_rate=config.learning_rate, beta_1=0.9, beta_2=0.999)
    model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])
    #early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3,restore_best_weights=True)


    train_generator, val_generator, test_generator = train_val__test_data_generation(batch_size=config.batch_size,augment_data=config.augment_data)

    hist=model.fit(train_generator, epochs=config.epochs, validation_data=val_generator, callbacks=[WandbCallback()])
    val_acc=max(hist.history['val_accuracy'])
    params={'batch_norm':config.batch_norm,'augmentation':config.augment_data,'dropout':config.dropout,
            'filter_architecture':config.filters,'kernel_size':config.kernel_size,'val_acc':val_acc}
    wandb.log(params)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910_DL_Assignment2", entity="nomads")

In [ ]:
wandb.agent('twsg746e', train, count = 50)